In [1]:
@dataclass
class Part:
    L: float      # m
    W: float      # m
    H: float      # m
    weight: float # kg

    @property
    def volume(self) -> float:
        return self.L * self.W * self.H

    @property
    def density(self) -> float:
        return self.weight / self.volume

    @property
    def slenderness(self) -> float:
        return self.L / min(self.W, self.H)

    def Q(self) -> float:
        S = self.slenderness
        rho = self.density
        return (S/5.0)**0.8 * (167.0/rho)**0.6


def exposure_km(Q: float, distance_km: float) -> float:
    """E = Q * (distance in thousand km)."""
    return Q * (distance_km / 1000.0)


def lambda_from_target(p_target: float, E_ref: float) -> float:
    """反解 λ：p = 1 - exp(-λ E)  =>  λ = -ln(1-p)/E."""
    if not (0 < p_target < 1):
        raise ValueError("p_target must be in (0,1)")
    return -log(1 - p_target) / E_ref


def route_damage_probability(
    Q: float,
    seg_km: Dict[str, float],         # {"road": km, "sea": km, "air": km}
    lambdas: Dict[str, float],        # {"road": λ_r, "sea": λ_s, "air": λ_a}
) -> float:
    Λ = 0.0
    for mode in ("road", "sea", "air"):
        d = seg_km.get(mode, 0.0)
        if d <= 0:
            continue
        E = exposure_km(Q, d)
        Λ += lambdas.get(mode, 0.0) * E
    return 1 - exp(-Λ)


# ---------- 本 case 的默认输入 ----------
# 尾灯：1.603×0.281×0.227 m，7.26 kg
part = Part(L=1.603, W=0.281, H=0.227, weight=7.26)
Q = part.Q()

# 三条路线的分段距离（km）
ROUTE_CN = {  # 佛山→盐田→(海)→洛杉矶→奥斯汀
    "road": 164 + 2221,  # 中国内陆卡车 + 美国内陆卡车
    "sea": 11962,
    "air": 0,
}
ROUTE_MX = {  # 蒙特雷→拉雷多过境→奥斯汀（只算路程；过境时间另算到交期）
    "road": 605,
    "sea": 0,
    "air": 0,
}
ROUTE_US = {  # Columbus, OH → Austin, TX
    "road": 1996,
    "sea": 0,
    "air": 0,
}

# 方式锚点（示例：你可按“可接受上限”自行设定）
# 举例思路：Road 2000 km 目标 1.5%；Sea 12000 km 目标 6%；Air 3000 km 目标 2%。
# 这些只是示例，改成你们的容忍度即可。
ANCHORS = {
    "road": {"p_ref": 0.015, "ref_km": 2000.0},
    "sea":  {"p_ref": 0.060, "ref_km": 12000.0},
    "air":  {"p_ref": 0.020, "ref_km": 3000.0},
}

# 反解各方式 λ
lambdas = {}
for mode, a in ANCHORS.items():
    E_ref = exposure_km(Q, a["ref_km"])
    lambdas[mode] = lambda_from_target(a["p_ref"], E_ref)

# 计算三条路线的损坏率
p_cn = route_damage_probability(Q, ROUTE_CN, lambdas)
p_mx = route_damage_probability(Q, ROUTE_MX, lambdas)
p_us = route_damage_probability(Q, ROUTE_US, lambdas)

print("=== Geometric factor ===")
print(f"Q = {Q:.3f}  | density = {part.density:.1f} kg/m^3 | slenderness = {part.slenderness:.2f}")
print("=== Lambdas by mode (per unit exposure) ===")
for k, v in lambdas.items():
    print(f"λ_{k} = {v:.5f}")

print("\n=== Damage probability by route ===")
print(f"China (road {ROUTE_CN['road']} km, sea {ROUTE_CN['sea']} km):   p = {p_cn:.4%}")
print(f"Mexico (road {ROUTE_MX['road']} km):                           p = {p_mx:.4%}")
print(f"USA (road {ROUTE_US['road']} km):                              p = {p_us:.4%}")

# ---- 可选：给出 1%–3% 的敏感区间（对每种方式分别上下调锚点）----
BAND = (0.01, 0.03)
def lambda_band(mode: str) -> Tuple[float, float]:
    lo, hi = BAND
    E_ref = exposure_km(Q, ANCHORS[mode]["ref_km"])
    return lambda_from_target(lo, E_ref), lambda_from_target(hi, E_ref)

λr_lo, λr_hi = lambda_band("road")
λs_lo, λs_hi = lambda_band("sea")
λa_lo, λa_hi = lambda_band("air")
print("\nLambda sensitivity bands (if p_ref in [1%, 3%] for each mode):")
print(f"road: [{λr_lo:.5f}, {λr_hi:.5f}]  sea: [{λs_lo:.5f}, {λs_hi:.5f}]  air: [{λa_lo:.5f}, {λa_hi:.5f}]")

=== Lambdas ===
  Distance: {'road': 0.00343, 'sea': 0.00234, 'air': 0.00306}
  Time    : {'road': 0.00182, 'sea': 0.00093, 'air': 0.00183}

[China  Foshan→Yantian→(SEA)→LA/LB→Austin]  Q=2.202  Λ=0.1205  p_damage=11.3486%
  - road: dist=  2385 km (E= 5.25), time= 1.00 d (E= 2.20),  contrib Λ=0.0220
  -  sea: dist= 11962 km (E=26.34), time=18.00 d (E=39.64),  contrib Λ=0.0984

[Mexico Monterrey→(Laredo)→Austin]  Q=2.202  Λ=0.0159  p_damage=1.5811%
  - road: dist=   605 km (E= 1.33), time= 2.83 d (E= 6.23),  contrib Λ=0.0159

[USA    Columbus,OH→Austin,TX]  Q=2.202  Λ=0.0231  p_damage=2.2850%
  - road: dist=  1996 km (E= 4.40), time= 2.00 d (E= 4.40),  contrib Λ=0.0231
